In [139]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [140]:
df = pd.read_pickle("./filtered_data.pkl")  
df.shape

(19816, 15)

In [141]:
df

,outcome_name,under_pressure,sub_type_name,body_part_name,shot_first_time,x,y,pass_deflected,shot_one_on_one,play_pattern_name,type_name,goalkeeper_position_name,technique_name,distance_to_goal,angle_to_goal
0,Blocked,NaN,Open Play,Right Foot,NaN,107.1,26.6,NaN,NaN,Regular Play,Shot,NaN,Normal,18.600269,15.850859
1,Blocked,NaN,Open Play,Right Foot,NaN,113.7,54.0,NaN,NaN,Regular Play,Shot,NaN,Normal,15.352199,11.720028
2,Off T,1.0,Open Play,Left Foot,True,115.7,50.5,NaN,NaN,Regular Play,Shot,NaN,Overhead Kick,11.346365,15.263642
3,Off T,NaN,Open Play,Head,NaN,113.9,38.6,NaN,NaN,From Throw In,Shot,NaN,Normal,6.258594,299.994760
4,Wayward,NaN,Open Play,Right Foot,NaN,96.7,47.2,NaN,NaN,From Free Kick,Shot,NaN,Half Volley,24.387087,16.351007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19811,Saved,NaN,Open Play,Right Foot,NaN,109.1,29.8,NaN,NaN,Regular Play,Shot,NaN,Normal,14.928161,20.853466
19812,Saved,NaN,Open Play,Right Foot,NaN,114.6,51.4,NaN,True,From Counter,Shot,NaN,Normal,12.614278,15.176401
19813,Blocked,NaN,Open Play,Right Foot,NaN,98.3,56.9,NaN,NaN,From Throw In,Shot,NaN,Normal,27.504545,12.065808
19814,Saved,NaN,Open Play,Left Foot,NaN,102.9,58.1,NaN,NaN,Regular Play,Shot,NaN,Normal,24.900201,11.658888


In [142]:
df["outcome"] = (df["outcome_name"] == "Goal").astype(int)
df

,outcome_name,under_pressure,sub_type_name,body_part_name,shot_first_time,x,y,pass_deflected,shot_one_on_one,play_pattern_name,type_name,goalkeeper_position_name,technique_name,distance_to_goal,angle_to_goal,outcome
0,Blocked,NaN,Open Play,Right Foot,NaN,107.1,26.6,NaN,NaN,Regular Play,Shot,NaN,Normal,18.600269,15.850859,0
1,Blocked,NaN,Open Play,Right Foot,NaN,113.7,54.0,NaN,NaN,Regular Play,Shot,NaN,Normal,15.352199,11.720028,0
2,Off T,1.0,Open Play,Left Foot,True,115.7,50.5,NaN,NaN,Regular Play,Shot,NaN,Overhead Kick,11.346365,15.263642,0
3,Off T,NaN,Open Play,Head,NaN,113.9,38.6,NaN,NaN,From Throw In,Shot,NaN,Normal,6.258594,299.994760,0
4,Wayward,NaN,Open Play,Right Foot,NaN,96.7,47.2,NaN,NaN,From Free Kick,Shot,NaN,Half Volley,24.387087,16.351007,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19811,Saved,NaN,Open Play,Right Foot,NaN,109.1,29.8,NaN,NaN,Regular Play,Shot,NaN,Normal,14.928161,20.853466,0
19812,Saved,NaN,Open Play,Right Foot,NaN,114.6,51.4,NaN,True,From Counter,Shot,NaN,Normal,12.614278,15.176401,0
19813,Blocked,NaN,Open Play,Right Foot,NaN,98.3,56.9,NaN,NaN,From Throw In,Shot,NaN,Normal,27.504545,12.065808,0
19814,Saved,NaN,Open Play,Left Foot,NaN,102.9,58.1,NaN,NaN,Regular Play,Shot,NaN,Normal,24.900201,11.658888,0


In [8]:

X = df[['body_part_name', 'sub_type_name', 'distance_to_goal', 'angle_to_goal']]
y = df['outcome']
X

,body_part_name,sub_type_name,distance_to_goal,angle_to_goal
0,Right Foot,Open Play,18.600269,15.850859
1,Right Foot,Open Play,15.352199,11.720028
2,Left Foot,Open Play,11.346365,15.263642
3,Head,Open Play,6.258594,299.994760
4,Right Foot,Open Play,24.387087,16.351007
...,...,...,...,...
19811,Right Foot,Open Play,14.928161,20.853466
19812,Right Foot,Open Play,12.614278,15.176401
19813,Right Foot,Open Play,27.504545,12.065808
19814,Left Foot,Open Play,24.900201,11.658888


In [9]:
X['body_part_name'], _ = pd.factorize(X['body_part_name'])  # Turn labels into classes
X['sub_type_name'], _ = pd.factorize(X['sub_type_name'])

C:\Users\Usuario\AppData\Local\Temp\ipykernel_1380\1906990171.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['body_part_name'], _ = pd.factorize(X['body_part_name'])  # Turn labels into classes
C:\Users\Usuario\AppData\Local\Temp\ipykernel_1380\1906990171.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['sub_type_name'], _ = pd.factorize(X['sub_type_name'])


In [143]:
### Standardization and blah blah blah

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
X_scaled

,body_part_name,sub_type_name,distance_to_goal,angle_to_goal
0,-0.826929,-0.223439,-0.056890,-0.640502
1,-0.826929,-0.223439,-0.448657,-0.670442
2,0.498190,-0.223439,-0.931823,-0.644758
3,1.823310,-0.223439,-1.545487,1.418982
4,-0.826929,-0.223439,0.641090,-0.636876
...,...,...,...,...
19811,-0.826929,-0.223439,-0.499803,-0.604242
19812,-0.826929,-0.223439,-0.778893,-0.645390
19813,-0.826929,-0.223439,1.017104,-0.667936
19814,0.498190,-0.223439,0.702979,-0.670885


In [11]:
# Remove outliers abs(std) > 3

X_cleaned = X_scaled[(X_scaled.abs() <= 3).all(axis=1)]
y_cleaned = y[(X_scaled.abs() <= 3).all(axis=1)]
X_cleaned

,body_part_name,sub_type_name,distance_to_goal,angle_to_goal
0,-0.826929,-0.223439,-0.056890,-0.640502
1,-0.826929,-0.223439,-0.448657,-0.670442
2,0.498190,-0.223439,-0.931823,-0.644758
3,1.823310,-0.223439,-1.545487,1.418982
4,-0.826929,-0.223439,0.641090,-0.636876
...,...,...,...,...
19811,-0.826929,-0.223439,-0.499803,-0.604242
19812,-0.826929,-0.223439,-0.778893,-0.645390
19813,-0.826929,-0.223439,1.017104,-0.667936
19814,0.498190,-0.223439,0.702979,-0.670885


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_cleaned, y_cleaned)

In [13]:
#Implementation of PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_train)
X_pca_test = pca.transform(X_test)

In [14]:
#Implementation of Logistic Regression
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=200).fit(X_train, y_train)
y_pred = clf.predict(X_test)
clf.score(X_test, y_test)

0.9036348267117498

In [15]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y[y_test.index], y_pred)

array([[4236,   13],
       [ 443,   40]])

In [16]:
#Implementation of Naive Bayes
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
bys = gnb.fit(X_train, y_train)
#el parameter de priors lo dejo por defecto porque es solo si la data doesen't portray el mundo real
#el de var_smoothing tampoco lo toco porque por default se usa este valor para prevenir overfitting

In [17]:
y_pred_gnb = bys.predict(X_test)
bys.score(X_test, y_test)

0.9030008453085376

In [18]:
confusion_matrix(y_test, y_pred_gnb)

array([[4238,   11],
       [ 448,   35]])

In [19]:
#Implementation of Decision Tree
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
treeclf = DecisionTreeClassifier(max_depth=25)
treeclf.fit(X_train, y_train)


,criterion,'gini'
,splitter,'best'
,max_depth,25
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [20]:
#treeplot = tree.plot_tree(treeclf)

In [21]:
from sklearn.metrics import accuracy_score
y_tree_predict = treeclf.predict(X_test)
acctpred = accuracy_score(y_test, y_tree_predict)
acctpred

0.9406170752324599

In [22]:
confusion_matrix(y_test, y_tree_predict)

array([[4120,  129],
       [ 152,  331]])

In [ ]:
#Implementation Ensamble Learing (hard voting)
from sklearn.ensemble import VotingClassifier
clf_hard_voting = VotingClassifier(
    estimators=[('lr', clf), ('gnb', bys), ('dt', treeclf)],
    voting = 'hard')
clf_hard_voting.fit(X_train, y_train)

,estimators,"[('lr', ...), ('gnb', ...), ...]"
,voting,'hard'
,weights,None
,n_jobs,None
,flatten_transform,True
,verbose,False
,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True


In [41]:
pred_hardvoting = clf_hard_voting.predict(X_test)
accuracy_hardvoting = accuracy_score(y_test,pred_hardvoting)
accuracy_hardvoting

0.9065934065934066

In [ ]:
def compare_classifiers_accuracies(clfs: List[object], X_train, X_test, y_train, y_test):
    for clf in clfs:
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        print(f"{clf.__class__.__name__}:", accuracy_score(y_test, y_pred))

compare_classifiers_accuracies([clf, bys, treeclf, clf_hard_voting], X_train, X_test, y_train, y_test)

LogisticRegression: 0.9036348267117498
GaussianNB: 0.9030008453085376
DecisionTreeClassifier: 0.9385038038884193
VotingClassifier: 0.9040574809805579


In [177]:
#Implementation Ensamble Learing (soft voting)
clf_soft_voting = VotingClassifier(
    estimators=[('lr', clf), ('gnb', bys), ('dt', treeclf)],
    voting = 'soft',
    weights= [2,2,4]) #added weightings to give more importance to the more accurate model
clf_soft_voting.fit(X_train, y_train)

,estimators,"[('lr', ...), ('gnb', ...), ...]"
,voting,'soft'
,weights,"[2, 2, ...]"
,n_jobs,None
,flatten_transform,True
,verbose,False
,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True


In [178]:
pred_softvoting = clf_soft_voting.predict(X_test)
accuracy_softvoting = accuracy_score(y_test,pred_softvoting)
compare_classifiers_accuracies([clf, bys, treeclf, clf_soft_voting], X_train, X_test, y_train, y_test)

LogisticRegression: 0.9036348267117498
GaussianNB: 0.9030008453085376
DecisionTreeClassifier: 0.9414623837700761
VotingClassifier: 0.9431530008453085


In [147]:
Pred_Proba_Ensamble = clf_soft_voting.predict_proba(X_test)
Pred_Proba_LogisticR = clf.predict_proba(X_test)
Pred_Proba_GNB = bys.predict_proba(X_test)
Pred_Proba_Tree = treeclf.predict_proba(X_test)

def Goal_Prob_Models(models=[clf, bys, treeclf,clf_soft_voting]):
    for model in models:
        print(f"{model.__class__.__name__}:",model.predict_proba(X_test))

Goal_Prob_Models()

LogisticRegression: [[0.76616329 0.23383671]
 [0.98529139 0.01470861]
 [0.97512777 0.02487223]
 ...
 [0.97099247 0.02900753]
 [0.90380082 0.09619918]
 [0.75958022 0.24041978]]
GaussianNB: [[0.97329756 0.02670244]
 [0.9984864  0.0015136 ]
 [0.99843889 0.00156111]
 ...
 [0.99099338 0.00900662]
 [0.98162337 0.01837663]
 [0.93941381 0.06058619]]
DecisionTreeClassifier: [[0.7037037 0.2962963]
 [1.        0.       ]
 [1.        0.       ]
 ...
 [1.        0.       ]
 [1.        0.       ]
 [0.        1.       ]]
VotingClassifier: [[0.76006271 0.23993729]
 [0.99558131 0.00441869]
 [0.99267063 0.00732937]
 ...
 [0.99042547 0.00957453]
 [0.96988929 0.03011071]
 [0.35122489 0.64877511]]


In [161]:
#Implementation Ensamble Learing (soft voting)
nonw_clf_soft_voting = VotingClassifier(
    estimators=[('lr', clf), ('gnb', bys)],
    voting = 'soft')
clf_soft_voting.fit(X_train, y_train)
compare_classifiers_accuracies([clf, bys, treeclf, nonw_clf_soft_voting], X_train, X_test, y_train, y_test)

LogisticRegression: 0.9036348267117498
GaussianNB: 0.9030008453085376
DecisionTreeClassifier: 0.9393491124260355
VotingClassifier: 0.9030008453085376
